## Imports

In [37]:
# Standard
import json
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

# Encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# Scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

# Models
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression

# Scoring
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from joblib import load


In [33]:
file_name = 'df'
df = pd.read_pickle(f'.\\processed_data\\{file_name}.pki')

In [34]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,5849,0.0,128.0,360.0,1.0,2,1
1,LP001003,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,LP001005,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,LP001006,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,LP001008,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


In [35]:
file_name = 'lr_model'
model = load(f'.\\ml_models\\{file_name}.joblib')

In [41]:
acc = 0.0
f1 = 0.0
iters = 100

for it in range(iters):
    #===================================General Processing====================================
    r = int.from_bytes(os.urandom(8), byteorder='little') % 4294967295
    train, test = train_test_split(df, test_size=.25, random_state=r)

    train['TotalIncome']=train['ApplicantIncome']+train['CoapplicantIncome']
    train['EMI']=train['LoanAmount']/train['Loan_Amount_Term']
    train['BalanceIncome']=train['TotalIncome']-train['EMI']

    train.drop(['ApplicantIncome','CoapplicantIncome','Loan_Amount_Term','LoanAmount'],axis=1,inplace=True)
    X_train, y_train = train.drop(columns=['Loan_ID', 'Loan_Status']), train['Loan_Status']

    test['TotalIncome']=test['ApplicantIncome']+test['CoapplicantIncome']
    test['EMI']=test['LoanAmount']/test['Loan_Amount_Term']
    test['BalanceIncome']=test['TotalIncome']-test['EMI']

    test.drop(['ApplicantIncome','CoapplicantIncome','Loan_Amount_Term','LoanAmount'],axis=1,inplace=True)
    X_test, y_test = test.drop(columns=['Loan_ID', 'Loan_Status']), test['Loan_Status']


    #========================================Logistic Regression===================================
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)


    # clf = LogisticRegression(solver='liblinear', penalty='l1', class_weight='balanced', C=0.11) # Liblinear
    clf = LogisticRegression(solver='saga', l1_ratio=0.1, penalty='elasticnet', class_weight='balanced', C=0.01) # Saga
    
    model = clf.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)


    #=======================================Support Vector Machine====================================
    

    # y_pred = model.predict(X_test)
    
    curr_f1 = f1_score(y_pred = y_pred, y_true=y_test)
    curr_acc = accuracy_score(y_pred = y_pred, y_true=y_test)

    f1 += curr_f1
    acc += curr_acc

    print(f'It: {it}')
    print(f'Acc: {curr_acc}')
    print(f'F1: {curr_f1}')



print(f'Accuracy of model: {file_name} is: {acc / iters}')
print(f'f1 score of model: {file_name} is: {f1 / iters}')


It: 0
Acc: 0.7727272727272727
F1: 0.8547717842323651
It: 1
Acc: 0.7792207792207793
F1: 0.8583333333333334
It: 2
Acc: 0.8311688311688312
F1: 0.8869565217391304
It: 3
Acc: 0.7792207792207793
F1: 0.8559322033898306
It: 4
Acc: 0.8116883116883117
F1: 0.8786610878661087
It: 5
Acc: 0.7987012987012987
F1: 0.8784313725490196
It: 6
Acc: 0.8311688311688312
F1: 0.8888888888888888
It: 7
Acc: 0.8116883116883117
F1: 0.8765957446808511
It: 8
Acc: 0.7597402597402597
F1: 0.8438818565400844
It: 9
Acc: 0.8116883116883117
F1: 0.8755364806866952
It: 10
Acc: 0.8116883116883117
F1: 0.8806584362139918
It: 11
Acc: 0.7792207792207793
F1: 0.859504132231405
It: 12
Acc: 0.7792207792207793
F1: 0.8508771929824562
It: 13
Acc: 0.8571428571428571
F1: 0.9105691056910569
It: 14
Acc: 0.7857142857142857
F1: 0.8630705394190871
It: 15
Acc: 0.8506493506493507
F1: 0.9037656903765691
It: 16
Acc: 0.7207792207792207
F1: 0.8138528138528138
It: 17
Acc: 0.7987012987012987
F1: 0.869198312236287
It: 18
Acc: 0.8506493506493507
F1: 0.902